## Demo: Log Automation

In [ ]:
import concurrent.futures
from arcgis.gis import GIS
from arcgis.gis.admin import PortalAdminManager
import datetime as _dt
from typing import Any

In [ ]:
gis:GIS = GIS(profile='your_dev_enterprise_profile')
admin:PortalAdminManager = gis.admin

#### Gather Portal Logs for Auditing 

In [ ]:
days_back:int
servers:dict[str, list[str,Any]] = {admin.url: []}
ref:dict[str, Any] = {admin.url: admin.logs}
start:_dt.datetime = _dt.datetime.now()

In [ ]:
jobs = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as tp:
    # gather the portal logs
    #
    days_back = admin.logs.settings.get("maxLogFileAge", 90)
    for d in range(days_back):
        future = tp.submit(admin.logs.query, 
                           **{"start_time" : start - _dt.timedelta(days=d), 
                              "end_time" : start - _dt.timedelta(days=d + 1)}
                          )
        jobs[future] = {
                        "server" : admin.url, 
                        'start_time' : start - _dt.timedelta(days=d), 
                        'end_time' : start - _dt.timedelta(days=d + 1)
                       }   
    # gather the server logs
    #
    for server in admin.servers.list():
        servers[server.url] = []
        ref[server.url] = server
        days_back = server.logs.settings.get("maxLogFileAge", 90)
        for d in range(days_back):
            future = tp.submit(server.logs.query, 
                               **{"start_time" : start - _dt.timedelta(days=d), 
                                  "end_time" : start - _dt.timedelta(days=d + 1)}
                              )
            jobs[future] = {
                            "server" : server.url, 
                            'start_time' : start - _dt.timedelta(days=d), 
                            'end_time' : start - _dt.timedelta(days=d + 1)
                           }   
    #  Push the log entries into the servers. 
    #
    for job in concurrent.futures.as_completed(jobs):
        try:
            records = job.result()
            servers[jobs[job]['server']].extend(records.get("logMessages", []))
        except:
            # retry the operation on 504 timeout
            params = jobs[job]
            server_url = params.pop('server')
            servers[server_url].extend(ref[server_url].logs.query(**params))

In [ ]:
for key in servers.keys():
    print(f"{key} : has {len(servers[key])} log records.")